In [8]:
import os
from glob import glob
import pandas as pd
import sqlite3

In [3]:
# List IR images
ir_images_filelist = glob('../nuevo_dataset/07_06_21_mask/*R*.jpg')
len(ir_images_filelist)

0

In [6]:
# Remove IR images
for image in ir_images_filelist:
    os.remove(image)

In [4]:
# List images
img_filelist = glob('../nuevo_dataset/07_06_21_mask/*.jpg')
len(img_filelist)

140

In [5]:
# Read database from thermal camera software
db = glob('../../../*.db')[0]
db

'../../..\\fd_DB.db'

In [9]:
cnx = sqlite3.connect(db)

In [10]:
df = pd.read_sql_query("SELECT * FROM faceinfo", cnx)

In [11]:
df.head()

,ftemp,img_path,temp_img_path,isAlarm,isWearMask,timeStamp
0,35.9,C:\Users\test\Documents\imagenes\image\2021-03...,C:\Users\test\Documents\imagenes\image\2021-03...,0,0,2021-03-24 09:03:34
1,35.9,C:\Users\test\Documents\imagenes\image\2021-03...,C:\Users\test\Documents\imagenes\image\2021-03...,0,0,2021-03-24 09:05:34
2,34.2,C:\Users\test\Documents\imagenes\image\2021-03...,C:\Users\test\Documents\imagenes\image\2021-03...,0,0,2021-03-24 09:05:55
3,37.1,C:\Users\test\Documents\imagenes\image\2021-03...,C:\Users\test\Documents\imagenes\image\2021-03...,0,1,2021-03-24 09:07:11
4,36.2,C:\Users\test\Documents\imagenes\image\2021-03...,C:\Users\test\Documents\imagenes\image\2021-03...,0,1,2021-03-24 09:07:39


In [12]:
df.describe()

,isAlarm,isWearMask
count,9069.000000,9069.000000
mean,0.368177,0.819936
std,0.776648,0.384835
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,3.000000,2.000000


In [17]:
start_date = "2021-06-07 00:00:00"

In [18]:
filtered_df = df.loc[df['timeStamp'] > start_date]

In [20]:
filtered_df.describe()

,isAlarm,isWearMask
count,1937.000000,1937.000000
mean,0.267940,0.870934
std,0.677997,0.336896
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,3.000000,2.000000


In [21]:
final_date = "2021-06-08 00:00:00"

In [22]:
filtered_df = filtered_df.loc[filtered_df['timeStamp'] < final_date]

In [23]:
filtered_df.describe()

,isAlarm,isWearMask
count,140.000000,140.000000
mean,0.257143,0.871429
std,0.671854,0.335927
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,2.000000,1.000000


In [47]:
filtered_df.loc[filtered_df.isWearMask==0].img_path.values

array(['C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\09-59-43_P_0_37.1.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-00-25_P_0_40.5.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-02-59_P_0_34.8.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-03-17_P_0_33.9.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-10-08_P_0_35.6.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-13-31_P_0_33.2.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-38-15_P_0_34.3.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-48-51_P_0_34.4.jpg',
       'C:\\Users\\test\\Documents\\imagenes\\image\\2021-06-07\\00.00~10.59\\tmp\\10-59-22_P_0_36.4.jpg',
       'C:\\Users\\test\\Documents\\i

In [32]:
import tensorflow as tf
from tensorflow import keras
from time import time
import numpy as np

In [34]:
# Define some parameters
dim = (160, 160)

In [36]:
# Load TF model
new_model = tf.keras.models.load_model('../models/keras_tuner.h5')

In [64]:
# Make predictions
label = 0
counter = 0
for file in img_filelist:
    tic=time()
    image=keras.preprocessing.image.load_img(file, target_size=dim)
    img_array = keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)
    predictions = np.squeeze(np.rint(new_model.predict(img_array))).astype('int')
    counter+=predictions
    if predictions==1:
        print(file)
    toc=time()-tic
    print("label: {}, prediction:{}, time(s): {}". format(label, predictions, toc))
print(counter)

label: 0, prediction:0, time(s): 0.0660855770111084
../nuevo_dataset/07_06_21_mask\06-23-32_P_0_34.8.jpg
label: 0, prediction:1, time(s): 0.05336499214172363
label: 0, prediction:0, time(s): 0.05596208572387695
../nuevo_dataset/07_06_21_mask\09-58-04_P_0_36.6.jpg
label: 0, prediction:1, time(s): 0.05183005332946777
label: 0, prediction:0, time(s): 0.052983999252319336
label: 0, prediction:0, time(s): 0.05803966522216797
../nuevo_dataset/07_06_21_mask\09-59-43_P_0_37.1.jpg
label: 0, prediction:1, time(s): 0.059682607650756836
label: 0, prediction:0, time(s): 0.05384111404418945
label: 0, prediction:0, time(s): 0.058287620544433594
label: 0, prediction:0, time(s): 0.055249929428100586
label: 0, prediction:0, time(s): 0.052680253982543945
label: 0, prediction:0, time(s): 0.05285787582397461
label: 0, prediction:0, time(s): 0.05748915672302246
label: 0, prediction:0, time(s): 0.056612253189086914
label: 0, prediction:0, time(s): 0.05149364471435547
label: 0, prediction:0, time(s): 0.053480

In [54]:
path = os.path.dirname(img_filelist[0])

In [49]:
failed_preds = filtered_df.loc[filtered_df.isWearMask==0].img_path.values

In [57]:
counter=0
for file in failed_preds:
#     print(os.path.join(path,os.path.basename(file)))
    image=keras.preprocessing.image.load_img(os.path.join(path,os.path.basename(file)), target_size=dim)
    img_array = keras.preprocessing.image.img_to_array(image)
    img_array = tf.expand_dims(img_array, 0)
    predictions = np.squeeze(np.rint(new_model.predict(img_array))).astype('int')
    print("label: {}, prediction:{}, time(s): {}". format(label, predictions, toc))
    counter+=predictions
print(counter)

label: 0, prediction:1, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:1, time(s): 0.0638284683227539
label: 0, prediction:1, time(s): 0.0638284683227539
label: 0, prediction:1, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:1, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
label: 0, prediction:0, time(s): 0.0638284683227539
5


In [61]:
100-counter/len(img_filelist)*100

90.71428571428571

In [63]:
100-len(filtered_df.loc[filtered_df.isWearMask==0].img_path.values)/len(img_filelist)*100

87.14285714285714